In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt

1. Define a function named get_lower_and_upper_bounds that has two arguments. 
    - The first argument is a pandas Series. 
    - The second argument is the multiplier, which should have a default argument of 1.5.

In [2]:
# define function to get upper and lower bounds
def get_lower_and_upper_bounds(series, multiplier=1.5):
    '''
    This function takes in a series and a multiplier (default=1.5). It returns
    lower and upper bounds using the IQR.
    '''
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3-q1
    lower = q1 - (multiplier*iqr)
    upper = q3 + (multiplier*iqr)
    return lower, upper

1. Using lemonade.csv dataset and focusing on continuous variables:

    - Use the IQR Range Rule and the upper and lower bounds to identify the lower outliers of each column of lemonade.csv, using the multiplier of 1.5. 
    - Do these lower outliers make sense? Which outliers should be kept?

In [3]:
# acquire data
lemonade = pd.read_csv('https://gist.githubusercontent.com/ryanorsinger/19bc7eccd6279661bd13307026628ace/raw/e4b5d6787015a4782f96cad6d1d62a8bdbac54c7/lemonade.csv')
lemonade.head()

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
0,1/1/17,Sunday,27.0,2.00,15,0.5,10
1,1/2/17,Monday,28.9,1.33,15,0.5,13
2,1/3/17,Tuesday,34.5,1.33,27,0.5,15
3,1/4/17,Wednesday,44.1,1.05,28,0.5,17
4,1/5/17,Thursday,42.4,1.00,33,0.5,18


In [4]:
# looking at datatypes
lemonade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         365 non-null    object 
 1   Day          365 non-null    object 
 2   Temperature  365 non-null    float64
 3   Rainfall     365 non-null    float64
 4   Flyers       365 non-null    int64  
 5   Price        365 non-null    float64
 6   Sales        365 non-null    int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 20.1+ KB


In [21]:
lemonade.describe().T

,count,mean,std,min,25%,50%,75%,max
Temperature,365.0,61.224658,18.085892,15.1,49.70,61.10,71.70,212.0
Rainfall,365.0,0.825973,0.273890,0.4,0.65,0.74,0.91,2.5
Flyers,365.0,40.104110,13.786445,-38.0,31.00,39.00,49.00,80.0
Price,365.0,0.500000,0.000000,0.5,0.50,0.50,0.50,0.5
Sales,365.0,27.865753,30.948132,7.0,20.00,25.00,30.00,534.0


In [10]:
# get lower and upper bounds for each numerical column
lower_Temperature, upper_Temperature = get_lower_and_upper_bounds(lemonade.Temperature)
lower_Rainfall, upper_Rainfall = get_lower_and_upper_bounds(lemonade.Rainfall)
lower_Flyers, upper_Flyers = get_lower_and_upper_bounds(lemonade.Flyers)
lower_Price, upper_Price = get_lower_and_upper_bounds(lemonade.Price)
lower_Sales, upper_Sales = get_lower_and_upper_bounds(lemonade.Sales)

In [11]:
# view lower outliers
pd.concat([lemonade[lemonade[col] < eval('lower_' + str(col))] for col in lemonade.select_dtypes(include=['int64', 'float64'])])

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
364,12/31/17,Sunday,15.1,2.50,9,0.5,7
324,11/21/17,Tuesday,47.0,0.95,-38,0.5,20


In [12]:
# function to get df of all outliers
def get_all_outliers(df, multiplier=1.5):
    '''
    This function takes in a dataframe and a multiplier (default=1.5) and returns
    a dataframe of both upper- and lower-bound outliers.
    '''
    outliers = pd.DataFrame()
    # Create a for loop that will iterate through the list of quantitative values calculating the IQR value
    # and setting the upper and lower limits. 
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        # Calculate value for IQR
        iqr = q3 - q1
        # Calculate the upper and lower limits for acceptable values(inclusive)
        inner_lower_limit = q1 - (multiplier * iqr)
        inner_upper_limit = q3 + (multiplier * iqr)
        outliers = outliers.append(df[df[col] < inner_lower_limit])
        outliers = outliers.append(df[df[col] > inner_upper_limit])
    return outliers
# test function
get_all_outliers(lemonade)

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
364,12/31/17,Sunday,15.1,2.50,9,0.5,7
41,2/11/17,Saturday,212.0,0.91,35,0.5,21
0,1/1/17,Sunday,27.0,2.00,15,0.5,10
1,1/2/17,Monday,28.9,1.33,15,0.5,13
2,1/3/17,Tuesday,34.5,1.33,27,0.5,15
5,1/6/17,Friday,25.3,1.54,23,0.5,11
6,1/7/17,Saturday,32.9,1.54,19,0.5,13
10,1/11/17,Wednesday,32.6,1.54,23,0.5,12
11,1/12/17,Thursday,38.2,1.33,16,0.5,14
12,1/13/17,Friday,37.5,1.33,19,0.5,15


In [13]:
# modify function to get only upper/lower outliers
def get_outliers(df, multiplier=1.5, bound='both'):
    '''
    This function takes in a dataframe, a multiplier (default=1.5), and a boundary
    direction ('upper', 'lower', default='both') and returns a dataframe of outliers
    based on which bound was specified.
    '''
    outliers = pd.DataFrame()
    if bound == 'both':
        # Create a for loop that will iterate through the list of quantitative values calculating the IQR value
        # and setting the upper and lower limits. 
        for col in df.select_dtypes(include=['int64', 'float64']).columns:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            # Calculate value for IQR
            iqr = q3 - q1
            # Calculate the upper and lower limits for acceptable values(inclusive)
            inner_lower_limit = q1 - (multiplier * iqr)
            inner_upper_limit = q3 + (multiplier * iqr)
            outliers = outliers.append(df[df[col] < inner_lower_limit])
            outliers = outliers.append(df[df[col] > inner_upper_limit])
    elif bound == 'lower':
        for col in df.select_dtypes(include=['int64', 'float64']).columns:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            # Calculate value for IQR
            iqr = q3 - q1
            # Calculate the upper and lower limits for acceptable values(inclusive)
            inner_lower_limit = q1 - (multiplier * iqr)
            outliers = outliers.append(df[df[col] < inner_lower_limit])
    elif bound == 'upper':
        for col in df.select_dtypes(include=['int64', 'float64']).columns:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            # Calculate value for IQR
            iqr = q3 - q1
            # Calculate the upper and lower limits for acceptable values(inclusive)
            inner_upper_limit = q3 + (multiplier * iqr)
            outliers = outliers.append(df[df[col] > inner_upper_limit])
    return outliers
# test function
get_outliers(lemonade, bound='lower')

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
364,12/31/17,Sunday,15.1,2.50,9,0.5,7
324,11/21/17,Tuesday,47.0,0.95,-38,0.5,20


Outlier on 12/31/17 is reasonable. 
However, the outlier on 11/21/17 has a negative value for Flyers, which doesn't make sense. Therefore, it makes sense to me to keep the outlier on 12/31/17 and to address the outlier on 11/21/17 by changing it to a positive number since that was probably a mistake in data entry.

### Use the IQR Range Rule and the upper and upper bounds to identify the upper outliers of each column of lemonade.csv, using the multiplier of 1.5. Do these upper outliers make sense? Which outliers should be kept?

In [15]:
# view upper outliers
pd.concat([lemonade[lemonade[col] > eval('upper_' + str(col))] for col in lemonade.select_dtypes(include=['int64', 'float64'])])

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
41,2/11/17,Saturday,212.0,0.91,35,0.5,21
0,1/1/17,Sunday,27.0,2.00,15,0.5,10
1,1/2/17,Monday,28.9,1.33,15,0.5,13
2,1/3/17,Tuesday,34.5,1.33,27,0.5,15
5,1/6/17,Friday,25.3,1.54,23,0.5,11
6,1/7/17,Saturday,32.9,1.54,19,0.5,13
10,1/11/17,Wednesday,32.6,1.54,23,0.5,12
11,1/12/17,Thursday,38.2,1.33,16,0.5,14
12,1/13/17,Friday,37.5,1.33,19,0.5,15
15,1/16/17,Monday,30.6,1.67,24,0.5,12


In [16]:
# use function to get upper outliers
get_outliers(lemonade, bound='upper')

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
41,2/11/17,Saturday,212.0,0.91,35,0.5,21
0,1/1/17,Sunday,27.0,2.00,15,0.5,10
1,1/2/17,Monday,28.9,1.33,15,0.5,13
2,1/3/17,Tuesday,34.5,1.33,27,0.5,15
5,1/6/17,Friday,25.3,1.54,23,0.5,11
6,1/7/17,Saturday,32.9,1.54,19,0.5,13
10,1/11/17,Wednesday,32.6,1.54,23,0.5,12
11,1/12/17,Thursday,38.2,1.33,16,0.5,14
12,1/13/17,Friday,37.5,1.33,19,0.5,15
15,1/16/17,Monday,30.6,1.67,24,0.5,12
